In [1]:
import pandas as pd
import sqlite3

## Task 1. Database Connection

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Task 2. New DFs

In [3]:
query = """
SELECT 
    time,
    AVG(user_avg_diff) as avg_diff
FROM (
    SELECT 
        uid,
        CASE 
            WHEN first_commit_ts < first_view_ts THEN 'before'
            ELSE 'after'
        END as time,
        AVG(CAST((julianday(first_commit_ts) - julianday(deadlines, 'unixepoch')) * 24 as INTEGER)) as user_avg_diff
    FROM test t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    AND t.uid IN (
        SELECT DISTINCT uid
        FROM test 
        WHERE labname != 'project1'
        GROUP BY uid
        HAVING 
            SUM(first_commit_ts < first_view_ts) > 0
            AND SUM(first_commit_ts >= first_view_ts) > 0
    )
    GROUP BY uid, time
)
GROUP BY time
"""

test_results = pd.io.sql.read_sql(query, conn)
test_results


,time,avg_diff
0,after,-99.523810
1,before,-66.047619


In [4]:
query = """
SELECT 
    time,
    AVG(user_avg_diff) as avg_diff
FROM (
    SELECT 
        uid,
        CASE 
            WHEN first_commit_ts < first_view_ts THEN 'before'
            ELSE 'after'
        END as time,
        AVG(CAST((julianday(first_commit_ts) - julianday(deadlines, 'unixepoch')) * 24 as INTEGER)) as user_avg_diff
    FROM control t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname != 'project1'
    AND t.uid IN (
        SELECT DISTINCT uid
        FROM control 
        WHERE labname != 'project1'
        GROUP BY uid
        HAVING 
            SUM(first_commit_ts < first_view_ts) > 0
            AND SUM(first_commit_ts >= first_view_ts) > 0
    )
    GROUP BY uid, time
)
GROUP BY time
"""
control_results = pd.io.sql.read_sql(query, conn)
control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


In [5]:
conn.close()

Гипотеза подтвердилась, тк Test группа пользователей показала значительное улучшение в статистике- стали начинать работу на 33.5 часов раньше